For for test its easiest to run BAML with the VS Code Extension!

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# Assuming your baml_client is inside a dir called app/
import sys
import os
sys.path.append(os.path.abspath("./baml_client"))

import baml_client as client # you can name this "llm" or "baml" or whatever you want
from baml_client import reset_baml_env_vars
import os
import dotenv

import sys

dotenv.load_dotenv()
reset_baml_env_vars(dict(os.environ))


from baml_client.types import ModuleCatalog


In [ ]:
with open('./baml_tests/mmds_newest_unstructured.txt', 'r') as file:
    text = file.read()
    # text = text[:10000]
len(text)

In [ ]:
catalog = client.b.ExtractModuleCatalog(text)
print(catalog)

In [ ]:
import os

def process_catalog_overview(folder_path):
    """
    Extracts the catalog overview with BAML from a text file and saves it as a JSON file.
    Args:
        folder_path (str): The path to the folder containing the input text file.
    """

    input_path = os.path.join(folder_path, 'merged_additional.txt')
    output_path = os.path.join(folder_path, 'catalog_overview.json')
    
    with open(input_path, 'r') as file:
        text = file.read()
    
    catalog = client.b.ExtractStudyProgrammOverview(text)
    
    catalog_json = catalog.model_dump_json()
    
    with open(output_path, 'w') as json_file:
        json_file.write(catalog_json)

# Example usage:
folder_path = './output/20241111_232155'
process_catalog_overview(folder_path)

[2024-11-11T22:38:10Z INFO  baml_events] Function ExtractStudyProgrammOverview:
    Client: openai/gpt-4o-mini (gpt-4o-mini-2024-07-18) - 27643ms. StopReason: stop. Tokens(in/out): 3346/2166
    ---PROMPT---
    [chat] system: Extract the information from this university module cataloge according to the defined schema:
    Master of Science (M.Sc.)
    “Mannheim Master in Data Science”
    University of Mannheim
    – Module catalog –
    for students starting in or after autumn 2024
    Academic Year
    HWS 2024/ FSS 2025Content
    Foreword ................................................................................................................................................. 3
    A. Overview ...................................................... 4
    B. Data Science Fundamentals ..................................... 5
    1. Overview .................................................................................................................................. 5
    2. 

In [ ]:
def process_module_list(input_path):
    """
    Extracts a list of modules with BAML from a text file and saves it as a JSON file.
    Args:
        input_path (str): The path to the input text file.
    """

    input_filename = os.path.basename(input_path)
    output_filename = f"{input_filename.split('.')[0]}_list.json"
    output_path = os.path.join(os.path.dirname(input_path), output_filename)
    
    with open(input_path, 'r') as file:
        text = file.read()
    
    module_list = client.b.ExtractModulesList(text)
    
    catalog_json = module_list.model_dump_json()
    
    with open(output_path, 'w') as json_file:
        json_file.write(catalog_json)

# Example usage:
input_path = 'output/20241111_232155/split_2_modules.txt'
process_module_list(input_path)


[2024-11-11T22:57:40Z INFO  baml_events] Function ExtractModulesList:
    Client: openai/gpt-4o-mini (gpt-4o-mini-2024-07-18) - 19335ms. StopReason: stop. Tokens(in/out): 2924/1259
    ---PROMPT---
    [chat] system: Extract the modules from this university module cataloge in the defined schema:
    2. Detailed descriptions
    CS 560 Large-Scale Data Management
    Form of module Lecture with exercises
    Type of module Computer Science Fundamental
    Level Master
    ECTS 6
    In presence: 56 h (4 SWS)
    Workload
    Exercises and self-study: 98 h
    Very good knowledge of database systems, good knowledge of al-
    Prerequisites
    gorithms and data structures as well as Java programming
    This course introduces the fundamental concepts and computa-
    tional paradigms of large-scale data management and Big Data.
    This includes methods for storing, updating, querying, and analyz-
    ing large dataset as well as for data-intensive computing. The
    course covers concep

In [ ]:
assert isinstance(catalog, ModuleCatalog)

In [ ]:
# save to json
catalog_json = catalog.model_dump_json()
print(catalog_json)
with open('./baml_tests/catalog.json', 'w') as json_file:
    json_file.write(catalog_json)

In [ ]:
# read from json
with open('./baml_tests/24-11-11_01_mmds_full_4omini.json', 'r') as json_file:
    catalog_json = json_file.read()

parsed_catalog = ModuleCatalog.model_validate_json(catalog_json)
print(parsed_catalog)
print(type(parsed_catalog))

In [ ]:
for i in range(len(parsed_catalog.studyAreas)):
    print(parsed_catalog.studyAreas[i].name)
    print(len(parsed_catalog.studyAreas[i].modules))

# should be: 4, 15 (14), 26 (14), 4, 12 (7), 18 (17)